In [82]:
#!pip install --pre python-docx
#!pip install pdfminer.six==20160614
#!pip install docx2txt
!pwd

/media/windows/1PropulsionAcademy/2019-10_HacknLead Preparation/Tools


In [70]:
path_to_example_pdf = '../challenge/case_files/US v Doss Article.pdf' #test1.pdf'

In [71]:
#PDF parsing libraries
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

In [72]:
#doc and docx parsing libraries
import docx2txt
import subprocess


In [73]:
import os


def parse_pdf(pdf_path):
    
    """
    :param filepath: path to document
    :return: string containing the parsed text
    """
    
    def extract_text_from_pdf(pdf_path):
        with open(pdf_path, 'rb') as fh:
            # iterate over all pages of PDF document
            for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
                # creating a resoure manager
                resource_manager = PDFResourceManager()

                # create a file handle
                fake_file_handle = io.StringIO()

                # creating a text converter object
                converter = TextConverter(
                                    resource_manager, 
                                    fake_file_handle, 
                                    codec='utf-8', 
                                    laparams=LAParams()
                            )

                # creating a page interpreter
                page_interpreter = PDFPageInterpreter(
                                    resource_manager, 
                                    converter
                                )

                # process current page
                page_interpreter.process_page(page)

                # extract text
                text = fake_file_handle.getvalue()
                yield text

                # close open handles
                converter.close()
                fake_file_handle.close()


    # calling above function and extracting text
    text = ''
    for page in extract_text_from_pdf(pdf_path):
        text += ' ' + page
    text = text.replace('\n', '')

    #raise NotImplementedError

    return text


def parse_docx(doc_path):

    """
    :param filepath: path to document
    :return: string containing the parsed text
    """
      
    temp = docx2txt.process(doc_path)
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
       
    #raise NotImplementedError

    return ' '.join(text)


def convert_doc_to_docx(filepath):

    """
    :param filepath: path to document
    :return: string containing the path of the converted document
    """
    
    subprocess.call(['soffice', '--headless', '--convert-to', 'docx', filepath, '--outdir', os.path.dirname(filepath)])
    path_docx = filepath + 'x'
    
    #raise NotImplementedError("Could not convert the file to .docx")

    return path_docx

In [74]:
def parse_file(filepath):

    """
    :param filepath: path to document
    :return: string containing the parsed text
    """

    filename_w_ext = os.path.basename(filepath)
    filename, file_extension = os.path.splitext(filename_w_ext)
    
    if file_extension == ".pdf":
        text = parse_pdf(filepath)

    elif file_extension == ".docx":
         text = parse_docx(filepath)

    elif file_extension == ".doc":
        filepath_docx = convert_doc_to_docx(filepath)
        text = parse_docx(filepath_docx)

    else:
        raise NotImplementedError("Unknown file format.")
    return text

In [94]:
!pwd

/media/windows/1PropulsionAcademy/2019-10_HacknLead Preparation/Tools


In [124]:
cases = []
for filename in os.listdir('/media/windows/1PropulsionAcademy/2019-10_HacknLead Preparation/challenge/case_files/'):
    if filename.endswith('.pdf'):
    #if __name__ == "__main__":
        try:
            example_pdf_path = '/media/windows/1PropulsionAcademy/2019-10_HacknLead Preparation/challenge/case_files/' + filename
            text_case = parse_file(example_pdf_path)
        except:  
            print('can not parse' + filename)
    cases.append(text_case)

can not parseUS v. Jimenez-Calderon - Antonia Judgement.pdf


In [99]:
name = example_pdf_path.split('/')[-1]

In [125]:
import pandas as pd

In [126]:
#cases[0]

In [131]:
cases_pd = pd.DataFrame(data = cases)

In [134]:
cases_pd.rename(columns={0:'case records'})

,case records
0,Toledo Man Sentenced to More than 10 Years in...
1,6/6/2018Area man charged with sex trafﬁcking ...
2,655585 SGOXXX10.1177/2158244016655585SAGE Ope...
3,"Servant was called an aunt, witness testifies..."
4,Nevada Man Sentenced to Life in Prison on Cha...
5,Toledo Man Sentenced to More than 10 Years in...
6,Toledo Man Sentenced to More than 10 Years in...


In [137]:
cases_pd.to_csv('court_cases_pdfs.csv', encoding='utf-8', index=False)
#df.to_csv('Data Scientist jobs.csv', encoding='utf-8', index=False)
